In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from itertools import product

pd.set_option('display.float_format', lambda x: '%.6f' % x)

def fetch_and_process_data(ticker, start_date="2023-01-01", end_date="2023-12-31", train_ratio=0.8):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty or "Adj Close" not in data:
            raise ValueError(f"No valid data found for {ticker}")
        
        close = data["Adj Close"]
        split_index = int(len(close) * train_ratio)
        
        price_scale = 1000 if close.mean() < 0.1 else 1
        scaled_close = close * price_scale
        
        # Use percentage returns if price is high enough, else use log returns
        returns = scaled_close.pct_change().dropna() if close.mean() > 0.1 else np.log(scaled_close / scaled_close.shift(1)).dropna()
        
        train_returns = returns.iloc[:split_index]
        test_returns = returns.iloc[split_index:]
        
        scaler = StandardScaler()
        scaler.fit(train_returns.values.reshape(-1, 1))
        
        train_std = scaler.transform(train_returns.values.reshape(-1, 1))
        test_std = scaler.transform(test_returns.values.reshape(-1, 1))
        
        std_returns = np.concatenate([train_std, test_std]).flatten()
        
        return close[returns.index], pd.Series(std_returns, index=returns.index), split_index
    
    except Exception as e:
        raise RuntimeError(f"Error processing {ticker}: {str(e)}")

def backtest_strategy(close, positions, entry_threshold=0.2, exit_threshold=-0.2):
    # Use thresholds to decide entries and exits
    entries = positions > entry_threshold
    exits = positions < exit_threshold
    pf = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        size=np.abs(positions),
        freq="1D"
    )
    return pf

# Strategy signal generators

def generate_signals_long_only(close, **kwargs):
    # Simple long-only: signal 1 if price increased from previous day
    signals = np.where(close.diff().fillna(0) > 0, 1, 0)
    return signals.astype(np.float64)

def generate_signals_macd(close, **kwargs):
    # MACD Calculation: difference between 12-day and 26-day EMA
    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signals = np.tanh(macd)  # normalize using tanh to be between -1 and 1
    return signals

def generate_signals_tsmom(returns, w=0, **kwargs):
    # TSMOM: scale the returns using a weight parameter
    signals = np.tanh(w * returns.values)
    return signals

def generate_signals_lstm_cpd(returns, split_index, lbw=21, epochs=50, **kwargs):
    # LSTM with CPD flavor: include a moving average as a second feature, computed over a lookback window (lbw)
    ma_feature = returns.rolling(window=lbw, min_periods=1).mean().values
    features = np.column_stack((returns.values, ma_feature))
    
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Prepare training data
    X_train = X[:split_index]
    train_features = features[:split_index]
    train_labels = y[:split_index].flatten()
    
    # Reshape features for LSTM input (samples, timesteps, features)
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    
    model = Sequential([
        Input(shape=(1, 2)),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    model.fit(train_features, train_labels, epochs=epochs, batch_size=32, verbose=0)
    
    predicted_signals = model.predict(all_features).flatten()
    return np.clip(predicted_signals, -1, 1)

def optimize_all_strategies(ticker="BTC-USD"):
    # Define strategy configurations with their respective parameter grids
    strategies = {
        "Long Only": {
            "generator": generate_signals_long_only,
            "params_grid": [{}]  
        },
        "MACD": {
            "generator": generate_signals_macd,
            "params_grid": [{}]
        },
        "TSMOM": {
            "generator": generate_signals_tsmom,
            "params_grid": [{'w': w} for w in [0, 0.5, 1]]
        },
        "LSTM w/ CPD": {
            "generator": generate_signals_lstm_cpd,
            "params_grid": [{'lbw': lbw, 'epochs': 50} for lbw in [10, 21, 63, 126, 252]]
        }
    }

    # Define grid for backtesting thresholds
    entry_thresholds = [0.15, 0.2, 0.25]
    exit_thresholds = [-0.15, -0.2, -0.25]

    close, returns, split_index = fetch_and_process_data(ticker)
    best_results = []

    for strat_name, strat_info in strategies.items():
        best_config = None
        best_return = -np.inf
        results = []
        print(f"\nOptimizing strategy: {strat_name}")

        for params in strat_info["params_grid"]:
            # Generate signals with the current strategy parameters
            if strat_name == "LSTM w/ CPD":
                signals = strat_info["generator"](returns, split_index, **params)
            elif strat_name in ["Long Only", "MACD"]:
                signals = strat_info["generator"](close, **params)
            else:
                signals = strat_info["generator"](returns, **params)

            # Combine each signal with every backtest threshold permutation
            for et, xt in product(entry_thresholds, exit_thresholds):
                pf = backtest_strategy(close, signals, entry_threshold=et, exit_threshold=xt)
                tot_ret = pf.total_return()
                sharpe = pf.sharpe_ratio()
                max_dd = pf.max_drawdown()

                config = {
                    'Strategy': strat_name,
                    **params,
                    'entry_threshold': et,
                    'exit_threshold': xt,
                    'Total Return': tot_ret,
                    'Sharpe Ratio': sharpe,
                    'Max Drawdown': max_dd
                }
                results.append(config)

                if tot_ret > best_return:
                    best_return = tot_ret
                    best_config = config

        results_df = pd.DataFrame(results)
        print(f"\n{strat_name} Grid Search Results:")
        print(results_df.to_markdown(index=False))
        print(f"\nBest {strat_name} Configuration:")
        print(best_config)

        best_results.append(best_config)

    # Find the best strategy across all permutations
    best_overall = sorted(best_results, key=lambda x: x['Total Return'], reverse=True)[0]
    print("\nBest Overall Strategy:", best_overall['Strategy'])
    print(best_overall)
    return best_results

def main():
    ticker = "BTC-USD"
    print(f"\nBacktesting {ticker} with full permutation optimization...")
    try:
        optimize_all_strategies(ticker)
    except Exception as e:
        print(f"Error in optimization: {str(e)}")

if __name__ == "__main__":
    main()



Backtesting BTC-USD with full permutation optimization...


[*********************100%%**********************]  1 of 1 completed



Optimizing strategy: Long Only

Long Only Grid Search Results:
| Strategy   |   entry_threshold |   exit_threshold |   Total Return |   Sharpe Ratio |   Max Drawdown |
|:-----------|------------------:|-----------------:|---------------:|---------------:|---------------:|
| Long Only  |              0.15 |            -0.15 |        1.49993 |        2.31797 |      -0.200578 |
| Long Only  |              0.15 |            -0.2  |        1.49993 |        2.31797 |      -0.200578 |
| Long Only  |              0.15 |            -0.25 |        1.49993 |        2.31797 |      -0.200578 |
| Long Only  |              0.2  |            -0.15 |        1.49993 |        2.31797 |      -0.200578 |
| Long Only  |              0.2  |            -0.2  |        1.49993 |        2.31797 |      -0.200578 |
| Long Only  |              0.2  |            -0.25 |        1.49993 |        2.31797 |      -0.200578 |
| Long Only  |              0.25 |            -0.15 |        1.49993 |        2.31797 |      -0.